# The Attention Mechanism - Interactive Learning Notebook

Welcome to the interactive attention mechanism tutorial! This notebook will guide you through implementing the core components of the attention mechanism step by step.

## Learning Objectives
By the end of this notebook, you will understand and implement:
1. **Linear Projections** for Query (Q), Key (K), and Value (V) matrices
2. **Scaled Dot-Product Attention** computation
3. **Softmax & Attention Weights** calculation
4. **Value Aggregation** using attention weights

## Example Prompt
Throughout this tutorial, we'll use this consistent example:

In [ ]:
PROMPT_EXAMPLE = "The cat sat on the mat"
print(f"Working with example: '{PROMPT_EXAMPLE}'")

## 📍 Notebook Navigation

- **Required** *(Graded)*: Sections 1-4 + Final Evaluation
- **Optional** *(Not Graded)*: Defined as "Part B" is optional for deeper understanding

⏱️ Estimated Time:
- **Required sections:** 45-60 minutes
- **Optional exploration:** 30+ minutes

## Setup and Imports
Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.visualizations import (
    visualize_qkv_projections,
    visualize_attention_scores,
    visualize_attention_weights,
    visualize_attended_values
)
from src.model_utils import tokenize_text, create_embeddings
from src.evaluation import evaluate_attention_output

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("Setup complete! All libraries imported successfully.")

---
# 📝 REQUIRED: Section 1: Linear Projections (Q, K, V)

## The Intuition: Three Perspectives on Information

Imagine you're at a library looking for information about "machine learning books." You would:
1. **Ask** the librarian about books on machine learning (Query - what you're looking for)
2. The librarian **checks** the catalog for available books (Keys - what's available to match)
3. You **receive** the actual books that match (Values - the information you get)

In attention mechanisms, we create these three "perspectives" for each word in our sentence.

## Theory: Why Do We Need Q, K, V?

The attention mechanism needs to answer: **"For each word, which other words should it pay attention to?"**

Consider our example: **"The cat sat on the mat"**

For the word "cat":
- **Query (Q)**: "What information does 'cat' need?" → Maybe it needs to know what action it's performing
- **Key (K)**: "What information can each word provide?" → "sat" can provide action information  
- **Value (V)**: "What is the actual information?" → The semantic meaning of "sat" (an action)

### The Three Transformations

Starting with the same input embeddings **X**, we create three different "views":

- **Query (Q)**: *"What am I looking for?"* - Transforms input to represent information needs
- **Key (K)**: *"What can I provide?"* - Transforms input to represent available information  
- **Value (V)**: *"What information do I actually contain?"* - Transforms input to represent the content to be retrieved

### Mathematical Formulation

$$Q = XW_Q$$
$$K = XW_K$$  
$$V = XW_V$$

Where:
- $X \in \mathbb{R}^{L \times d_{model}}$: Input embeddings (sequence length × embedding dimension)
- $W_Q, W_K, W_V \in \mathbb{R}^{d_{model} \times d_k}$: Learned weight matrices (embedding dim × projection dim)
- $Q, K, V \in \mathbb{R}^{L \times d_k}$: Projected query, key, value matrices

### Why Different Weight Matrices?

Each weight matrix learns to extract different aspects:
- $W_Q$: Learns to extract "what information this position needs"
- $W_K$: Learns to extract "what information this position can provide" 
- $W_V$: Learns to extract "the actual information content"

### Tensor Shape Deep Dive

For "The cat sat on the mat" (6 tokens):
- Input embeddings: `(1, 6, 512)` → 1 batch, 6 tokens, 512-dim embeddings
- After projection: `(1, 6, 64)` → 1 batch, 6 tokens, 64-dim projections

The reduction from 512 to 64 dimensions serves two purposes:
1. **Computational efficiency**: Smaller attention computations
2. **Multiple heads**: We can have multiple attention heads in parallel

# PART A: CORE ASSIGNMENT (GRADED)

In [ ]:
# Initialize example data
PROMPT_EXAMPLE = "The cat sat on the mat"
tokens = tokenize_text(PROMPT_EXAMPLE, method='word')  # Use word-level tokenization to match expected format
embeddings = create_embeddings(tokens)
print(f"Tokens: {tokens}")
print(f"Embedding shape: {embeddings.shape}")

In [ ]:
def create_qkv_projections(embeddings, d_model=512, d_k=64):
    """
    Create Query, Key, and Value projections from input embeddings.
    
    Args:
        embeddings: Input embeddings tensor (batch_size, seq_len, d_model)
        d_model: Dimension of input embeddings
        d_k: Dimension of Q, K, V projections
    
    Returns:
        Q, K, V: Query, Key, Value tensors (batch_size, seq_len, d_k)
    """
    
    # ===== REQUIRED IMPLEMENTATION (GRADED) =====
    # TODO: Get input dimensions
    batch_size, seq_len, embedding_dim = embeddings.shape
    # TODO: Create linear projection layers
    W_q = nn.Linear(embedding_dim, d_k, bias=False)
    W_k = nn.Linear(embedding_dim, d_k, bias=False)
    W_v = nn.Linear(embedding_dim, d_k, bias=False)
    # TODO: Apply projections to input embedings
    Q = W_q(embeddings)  # (batch_size, seq_len, d_k)
    K = W_k(embeddings)  # (batch_size, seq_len, d_k)
    V = W_v(embeddings)  # (batch_size, seq_len, d_k)
    # ===== END REQUIRED SECTION =====
    return Q, K, V

# Test implementation
Q, K, V = create_qkv_projections(embeddings)
print(f"Q shape: {Q.shape}, K shape: {K.shape}, V shape: {V.shape}")
print("Successfully created Q, K, V projections!")

In [ ]:
# VISUALIZATION
visualize_qkv_projections(embeddings, Q, K, V, tokens)

---
# 📝 REQUIRED: Section 2: Scaled Dot-Product Attention

## The Intuition: Measuring Compatibility

Think of this step as **matchmaking between questions and answers**:
- Each Query asks: *"What information do I need?"*
- Each Key responds: *"Here's what I can provide"*  
- The dot product measures: *"How well do they match?"*

### Why Dot Product for Similarity?

The dot product between two vectors measures their **alignment**:
- **High dot product**: Vectors point in similar directions → High compatibility
- **Low dot product**: Vectors are orthogonal → Low compatibility  
- **Negative dot product**: Vectors point in opposite directions → Incompatible

**Example**: If Query for "cat" is looking for "action information" and Key for "sat" provides "action information", their dot product will be high.

### The Mathematical Operation

$$\text{Attention Scores} = \frac{QK^T}{\sqrt{d_k}}$$

Let's break this down step by step:

#### Step 1: Matrix Multiplication $QK^T$
- $Q \in \mathbb{R}^{L \times d_k}$: Each row is a query vector for one token
- $K^T \in \mathbb{R}^{d_k \times L}$: Each column is a key vector for one token  
- Result: $\mathbb{R}^{L \times L}$ matrix where entry $(i,j)$ = similarity between token $i$'s query and token $j$'s key

#### Step 2: Scaling by $\sqrt{d_k}$

**Why do we need scaling?**
As the dimension $d_k$ increases, dot products tend to grow larger in magnitude. This pushes values toward the extremes of the softmax function where gradients become extremely small.

**The Problem**: Without scaling, for $d_k = 512$:
- Random dot products have variance ≈ 512
- Softmax becomes nearly deterministic (almost one-hot)
- Gradients vanish during training

**The Solution**: Dividing by $\sqrt{d_k}$ normalizes the variance back to ≈ 1

### Tensor Shape Analysis

For "The cat sat on the mat" (6 tokens, $d_k = 64$):

1. **Q shape**: `(1, 6, 64)` - 6 query vectors, each 64-dimensional
2. **K shape**: `(1, 6, 64)` - 6 key vectors, each 64-dimensional  
3. **K^T shape**: `(1, 64, 6)` - Transposed for matrix multiplication
4. **QK^T shape**: `(1, 6, 6)` - 6×6 attention score matrix

Each element `[i, j]` represents: *"How much should token i attend to token j?"*

### Attention Score Matrix Interpretation

For our example sentence, the 6×6 matrix might look like:
```
         The  cat  sat  on  the  mat
    The  [ ?   ?    ?   ?   ?    ? ]
    cat  [ ?   ?    ?   ?   ?    ? ]  
    sat  [ ?   ?    ?   ?   ?    ? ]
    on   [ ?   ?    ?   ?   ?    ? ]
    the  [ ?   ?    ?   ?   ?    ? ]
    mat  [ ?   ?    ?   ?   ?    ? ]
```

Higher scores indicate stronger relationships (e.g., "cat" → "sat" for subject-verb relationship).

### The Complete Formula Intuition

$$\text{Score}_{i,j} = \frac{\text{query}_i \cdot \text{key}_j}{\sqrt{d_k}}$$

This answers: *"How relevant is the information that token j can provide to what token i is looking for?"*

In [ ]:
def compute_attention_scores(Q, K):
    """
    Compute scaled dot-product attention scores.
    
    Args:
        Q: Query tensor (batch_size, seq_len, d_k)
        K: Key tensor (batch_size, seq_len, d_k)
    
    Returns:
        attention_scores: Attention scores (batch_size, seq_len, seq_len)
    """
    # Implementation follows: scores = Q * K^T / sqrt(d_k)
    
    # ===== REQUIRED IMPLEMENTATION (GRADED) =====
    # TODO: Get the dimension d_k for scaling
    d_k = K.shape[-1]
    # TODO: Compute dot product between Q and K^T
    # Q: (batch_size, seq_len, d_k)
    # K^T: (batch_size, d_k, seq_len)
    attention_scores = torch.matmul(Q, K.transpose(-2, -1))  # (batch_size, seq_len, seq_len)
    # TODO: Scale by sqrt(d_k) to prevent extreme values
    attention_scores = attention_scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
    # ===== END REQUIRED SECTION =====
    
    return attention_scores

# Test implementation
attention_scores = compute_attention_scores(Q, K)
print(f"Attention scores shape: {attention_scores.shape}")
print(f"d_k = {Q.shape[-1]}, scaling factor = {torch.sqrt(torch.tensor(Q.shape[-1], dtype=torch.float32)):.2f}")
print("Successfully computed attention scores!")

In [ ]:
# VISUALIZATION
visualize_attention_scores(attention_scores, tokens)

---
# 📝 REQUIRED: Section 3: Softmax & Attention Weights

## The Intuition: From Scores to Decisions

Imagine you're deciding how to allocate your attention while reading "The cat sat on the mat":
- You have **compatibility scores** for how relevant each word is
- But you need to make a **decision**: How much attention to give each word?
- Softmax converts raw scores into a **probability distribution** - a recipe for attention allocation

### Why Convert to Probabilities?

Raw attention scores can be any real numbers (positive, negative, large, small). We need:
1. **Interpretability**: Weights between 0 and 1 are easier to understand
2. **Normalization**: Weights sum to 1, so we're not "over-attending"  
3. **Differentiability**: Smooth function for gradient-based learning

### The Softmax Function

$$\text{Attention Weight}_{i,j} = \frac{\exp(\text{Score}_{i,j})}{\sum_{k=1}^{L} \exp(\text{Score}_{i,k})}$$

**What this does**:
- **Exponential**: $\exp(x)$ makes all values positive and amplifies differences
- **Normalization**: Division ensures weights sum to 1 for each query position
- **Probability distribution**: Each row becomes a valid probability distribution

### Step-by-Step Example

For "The cat sat on the mat", let's say token "cat" has attention scores:
```
Raw scores:     [0.1, 0.8, 1.2, 0.3, 0.1, 0.4]
After exp():    [1.11, 2.23, 3.32, 1.35, 1.11, 1.49]
Sum:            11.61
After softmax:  [0.09, 0.19, 0.29, 0.12, 0.09, 0.13]
```

**Interpretation**: "cat" should pay:
- 29% attention to "sat" (highest score → highest weight)
- 19% attention to itself  
- 13% attention to "mat"
- etc.

### The Attention Matrix

After applying softmax to all rows, we get a **stochastic matrix**:

$$\text{Attention}_{6 \times 6} = \begin{bmatrix}
\text{The→The} & \text{The→cat} & \text{The→sat} & \cdots \\
\text{cat→The} & \text{cat→cat} & \text{cat→sat} & \cdots \\
\text{sat→The} & \text{sat→cat} & \text{sat→sat} & \cdots \\
\vdots & \vdots & \vdots & \ddots
\end{bmatrix}$$

**Properties**:
- Each row sums to 1 (probability distribution)
- Each entry is between 0 and 1
- Row $i$ shows how token $i$ distributes its attention

### Concrete Example: "The cat sat on the mat"

The attention weights might reveal linguistic patterns:
```
         The   cat   sat   on    the   mat
    The [0.2, 0.15, 0.1, 0.15, 0.25, 0.15]  # Articles attend to nouns
    cat [0.1, 0.25, 0.4, 0.05, 0.05, 0.15]  # Subject attends to verb
    sat [0.05, 0.35, 0.3, 0.1, 0.05, 0.15]  # Verb attends to subject
    on  [0.1, 0.1, 0.15, 0.2, 0.15, 0.3]   # Preposition attends to object
    the [0.15, 0.1, 0.1, 0.15, 0.25, 0.25]  # Article attends to noun
    mat [0.1, 0.2, 0.15, 0.25, 0.15, 0.15]  # Object attends to preposition
```

**Key Insights**:
- "cat" (row 2) has highest weight 0.4 for "sat" → Subject-verb relationship
- "on" (row 4) has highest weight 0.3 for "mat" → Preposition-object relationship
- Self-attention captures word importance in context

### Mathematical Properties

1. **Row-wise normalization**: $\sum_{j=1}^{L} \text{Attention}_{i,j} = 1$ for all $i$

2. **Temperature effect**: Higher scores get exponentially more weight
   - Score difference of 1 → Weight ratio of $e ≈ 2.7$
   - Score difference of 2 → Weight ratio of $e^2 ≈ 7.4$

3. **Concentration**: Softmax concentrates probability mass on highest scores

In [ ]:
# COMPLETED: Implement softmax to get attention weights
# Implemented: Apply softmax to convert attention scores to attention weights

def compute_attention_weights(attention_scores):
    """
    Convert attention scores to attention weights using softmax.
    
    Args:
        attention_scores: Attention scores (batch_size, seq_len, seq_len)
    
    Returns:
        attention_weights: Attention weights (batch_size, seq_len, seq_len)
    """
    # Implementation follows: weights = softmax(scores)
    
    # ===== REQUIRED IMPLEMENTATION (GRADED) =====
    # TODO: Apply softmax along the last dimension (key positions)
    attention_weights = F.softmax(attention_scores, dim=-1)
    # ===== END REQUIRED SECTION =====
    
    return attention_weights

# Test implementation
attention_weights = compute_attention_weights(attention_scores)
print(f"Attention weights shape: {attention_weights.shape}")
print(f"Sum of weights for first query position: {attention_weights[0, 0, :].sum():.4f}")
print(f"Sum of weights for second query position: {attention_weights[0, 1, :].sum():.4f}")
print("All attention weights should sum to 1.0 for each query position")
print("Successfully computed attention weights!")

In [ ]:
# VISUALIZATION
visualize_attention_weights(attention_weights, tokens)

---
# 📝 REQUIRED: Section 4: Value Aggregation

## The Intuition: Gathering Information

Now comes the payoff! We've decided **where** to look (attention weights), now we need to **gather** the actual information from those locations. This is like:

- **Step 3 of our library analogy**: After deciding which books are most relevant (attention weights), you actually **read and combine** information from those books (values)
- **Weighted averaging**: Instead of reading all books equally, you focus more on the most relevant ones

### The Mathematical Operation

$$\text{Output} = \text{Attention Weights} \times V$$

More precisely:
$$\text{Output}_i = \sum_{j=1}^{L} \text{Attention}_{i,j} \times V_j$$

Where:
- $\text{Output}_i$: The new representation for token $i$
- $\text{Attention}_{i,j}$: How much token $i$ attends to token $j$  
- $V_j$: The value vector for token $j$

### Conceptual Understanding

For each token, we create a **personalized summary** of the entire sequence:

**For token "cat" in "The cat sat on the mat":**
```
Original value of "cat": [cat's semantic features]
After attention:         [0.1×"The" + 0.25×"cat" + 0.4×"sat" + 0.05×"on" + 0.05×"the" + 0.15×"mat"]
```

**The result**: "cat" now contains:
- 40% of "sat"'s information (strong subject-verb connection)  
- 25% of its own information (self-context)
- 15% of "mat"'s information (object relationship)
- Small amounts from other tokens

### What Makes This Powerful?

1. **Contextualization**: Each token's representation now includes relevant context
2. **Selective Focus**: More important relationships get more weight
3. **Information Flow**: Semantic information flows from keys to queries through values

### Tensor Shape Analysis

For "The cat sat on the mat" (6 tokens, $d_k = 64$):

1. **Attention weights**: `(1, 6, 6)` - How each token attends to every other token
2. **Values (V)**: `(1, 6, 64)` - 64-dimensional value vector for each token
3. **Output**: `(1, 6, 64)` - 64-dimensional attended representation for each token

**Matrix multiplication**:
- Row $i$ of attention weights: `(1, 6)` - attention distribution for token $i$
- Full values matrix: `(6, 64)` - all value vectors
- Result for token $i$: `(1, 64)` - weighted combination of all value vectors

### The Complete Information Flow

Let's trace what happens to the word "cat":

1. **Query Creation**: "cat" → Query vector (what information does "cat" need?)
2. **Attention Computation**: Query compared to all Key vectors → Attention scores  
3. **Softmax**: Attention scores → Attention weights (probability distribution)
4. **Value Aggregation**: Attention weights × Value vectors → Final representation

**The result**: The new representation of "cat" contains:
- Its original semantic information
- **Plus** contextual information from "sat" (it performs this action)
- **Plus** contextual information from "mat" (location relationship)  
- **Plus** smaller amounts from other tokens

### Why Values Are Different From Keys?

- **Keys**: Optimized to be "found" by queries (searchable representations)
- **Values**: Optimized to provide useful information (retrievable content)
- **Analogy**: Keys are like book titles/tags, Values are like book contents

### Example: Attention in Practice

For "The cat sat on the mat":

**Before attention**: Each word has only its own meaning
- "cat" → [animal, feline, small, ...]
- "sat" → [action, past tense, positioning, ...]

**After attention**: Each word incorporates contextual information  
- "cat" → [animal, feline, **performed sitting**, **on furniture**, ...]
- "sat" → [action, **done by cat**, past tense, **on mat**, ...]

### The Output: Contextualized Representations

The final output is a set of **contextualized embeddings** where each token's representation has been enriched with relevant information from the entire sequence, weighted by attention.

This forms the foundation for:
- **Language understanding**: Words understand their context
- **Compositionality**: Meaning emerges from relationships  
- **Long-range dependencies**: Distant words can influence each other

In [ ]:
def aggregate_values(attention_weights, V):
    """
    Aggregate value vectors using attention weights.
    
    Args:
        attention_weights: Attention weights (batch_size, seq_len, seq_len)
        V: Value tensor (batch_size, seq_len, d_v)
    
    Returns:
        output: Attended output (batch_size, seq_len, d_v)
    """
    # ===== REQUIRED IMPLEMENTATION (GRADED) =====
    # TODO: Multiply attention weights with value vectors
    # attention_weights: (batch_size, seq_len, seq_len)
    # V: (batch_size, seq_len, d_v)
    # output: (batch_size, seq_len, d_v)
    output = torch.matmul(attention_weights, V)
    # ===== END REQUIRED SECTION =====
    
    return output

# Test implementation
attended_output = aggregate_values(attention_weights, V)
print(f"Attended output shape: {attended_output.shape}")
print(f"Original V shape: {V.shape}")
print(f"Attention weights shape: {attention_weights.shape}")
print("Successfully computed attended values!")
print("Each token now has a contextualized representation!")

In [ ]:
# VISUALIZATION
visualize_attended_values(attended_output, attention_weights, tokens)

---
# Complete Attention Mechanism

You did it, great job!
Now let's put it all together into a complete attention function

In [ ]:
def attention_mechanism(embeddings, d_k=64):
    """
    Complete attention mechanism implementation.
    
    Args:
        embeddings: Input embeddings (batch_size, seq_len, d_model)
        d_k: Dimension for Q, K, V projections
    
    Returns:
        output: Attended output (batch_size, seq_len, d_k)
        attention_weights: Attention weights for visualization
    """
    # Create Q, K, V projections
    Q, K, V = create_qkv_projections(embeddings, d_k=d_k)
    
    # Compute attention scores
    attention_scores = compute_attention_scores(Q, K)
    
    # Apply softmax to get attention weights
    attention_weights = compute_attention_weights(attention_scores)
    
    # Aggregate values using attention weights
    output = aggregate_values(attention_weights, V)
    
    return output, attention_weights

# Test complete implementation
final_output, final_attention_weights = attention_mechanism(embeddings)
print(f"Final output shape: {final_output.shape}")
print(f"Final attention weights shape: {final_attention_weights.shape}")
print(f"Input embedding dim: {embeddings.shape[-1]}, Output dim: {final_output.shape[-1]}")
print("Complete attention mechanism implemented successfully!")
print("The attention mechanism has transformed static embeddings into contextualized representations!")

In [ ]:
# Evaluation of your implementation
evaluation_results = evaluate_attention_output(final_output, final_attention_weights, embeddings)
print("Evaluation Results:")
for key, value in evaluation_results.items():
    print(f"{key}: {value}")

# ═══════════════════════════════════════════════════════════════
# END OF GRADED ASSIGNMENT
# HERE BEGINS PART B: DEEPER UNERSTANDING

You are officially done with the mandatory lesson and first of all, awesome work completing sections 1-4 and thank you for taking my lesson!
It's difficult to cover topics as large as  *Attention & Transformers* in such a short period of time.
I have therefore decided to add **optional** content below for those that want to keep learning.
This requires no further interaction from the student - simply sit back, relax and run some cells.
# ═══════════════════════════════════════════════════════════════

# 🔬 Section 5: Transformer Model Comparison

## From Educational Implementation to Production Reality

Now that you understand how attention works from first principles, let's see how your implementation compares to real-world production transformers. This section bridges the gap between learning and practical application.

### The Scale Gap: Education vs Production

Your reference implementation was designed for **clarity and understanding**:
- 64-dimensional embeddings (easy to visualize and debug)
- Single attention head (focus on core mechanism)
- One attention computation (minimal complexity)
- Educational example: "The cat sat on the mat"

Production transformers prioritize **performance and expressiveness**:
- 768+ dimensional embeddings (rich representation space)
- 12+ attention heads (multiple perspectives on relationships)
- 6-12+ layers (deep hierarchical processing)
- Complex tokenization and vocabulary handling

### Key Questions This Section Answers

1. **Scale**: How much larger are production models compared to your reference?
2. **Architecture**: What additional complexity do production models add?
3. **Consistency**: Do production models use the same core attention mechanism?
4. **Performance**: Why do production models need so much more complexity?

### Your Learning Journey

This comparison helps you:
- **Appreciate the fundamentals**: The core mechanism remains the same
- **Understand complexity**: See why production models are more sophisticated
- **Bridge theory to practice**: Connect academic understanding to real applications
- **Gain perspective**: Recognize what scales and what stays constant

Let's explore these differences hands-on!

In [ ]:
# ORDER: #3
# TODO: Compare your implementation with production transformer

# Import the comparison function
from src.model_utils import compare_attention_implementations

print("Running side-by-side comparison...")
print("=" * 60)

# Run the comprehensive comparison using our example text
try:
    comparison_results = compare_attention_implementations(PROMPT_EXAMPLE)
    
    print("\nComparison Summary:")
    print(f"Reference embedding dimension: {comparison_results['comparison']['embedding_dimensions']['reference']}")
    print(f"Transformer embedding dimension: {comparison_results['comparison']['embedding_dimensions']['transformer']}")
    print(f"Dimension ratio (production/reference): {comparison_results['comparison']['embedding_dimensions']['ratio']:.1f}x")
    print(f"Both use proper attention normalization: {comparison_results['comparison']['attention_patterns']['both_sum_to_one']}")
    
    print("\nKey Educational Insights:")
    for insight in comparison_results['educational_insights']:
        print(f"• {insight}")
    
    print("\nComparison complete! Ready for visualization.")
        
except Exception as e:
    print(f"Comparison failed: {e}")
    print("This might be due to missing model loading or implementation issues")
    comparison_results = None

## 🔬 Visual Comparison

Now that we have comparison results, let's create visualizations that show the differences between your reference implementation and the production transformer. This will help you see both the similarities and differences at a glance.

### What These Visualizations Show

1. **Attention Weight Heatmaps**: Side-by-side comparison of attention patterns
2. **Embedding Dimension Comparison**: Visual representation of the scale difference
3. **Architecture Complexity**: Comparison of model complexity metrics
4. **Parameter Count Visualization**: Understanding the computational requirements

These visualizations make abstract concepts concrete and help bridge the gap between educational simplicity and production complexity.

### Interpreting the Visualizations

When you look at the visualizations:

1. **Attention Patterns**: Notice how both use similar probability distributions
2. **Dimension Scale**: See the 12x size difference visually represented
3. **Complexity**: Observe the difference between 1 head vs 12 heads
4. **Core Mechanism**: Both use the same fundamental attention formula

In [ ]:
# ORDER: #5
# TODO: Visualize model comparison

# Import the visualization function (if not already imported)
from src.model_utils import visualize_model_comparison

print("Creating visual comparison of implementations...")
print("Note: This creates a comprehensive 2x2 subplot showing key differences")

# Use the visualize_model_comparison() function
if 'comparison_results' in locals() and comparison_results is not None:
    try:
        visualize_model_comparison(comparison_results)
        print("Visualization created successfully!")
        print("Compare the attention patterns, dimensions, and architecture complexity.")
    except Exception as e:
        print(f"Visualization error: {e}")
        print("This may be due to matplotlib backend issues in some environments")
else:
    print("No comparison results available. Please run the comparison cell first.")

## 🔬 Dimension Adaptation: Bridging the Gap

One challenge when working with both educational and production models is handling the dimension mismatch. Your reference uses 64D embeddings while production models use 768D+. Let's explore how to bridge this gap.

### Why Dimension Adaptation Matters

- **Integration**: Combining insights from both implementations
- **Visualization**: Adapting production outputs for educational visualization
- **Experimentation**: Testing ideas across different scales
- **Understanding**: Seeing how dimensional choices affect model behavior

### Adaptation Methods

1. **Projection**: Linear transformation (learns optimal mapping)
2. **Padding**: Adding zeros (preserves original information)
3. **Truncation**: Simple reduction (may lose information)

### TODO: Experiment with Adaptation

Consider these questions as you explore:

1. **Which method** do you think would work best for different use cases?
2. **What information** might be lost when reducing dimensions?
3. **How might** different adaptation methods affect attention patterns?
4. **When would you** want to go from 64D to 768D vs 768D to 64D?

In [ ]:
# ORDER: #7
# COMPLETED: Explore dimension adaptation

# Import the adapt_dimensions function
from src.model_utils import adapt_dimensions

print("Exploring dimension adaptation between reference and production scales...")
print("=" * 70)

# Get your reference embeddings from earlier in the notebook
print(f"Starting with reference embeddings: {embeddings.shape}")

# Experiment with dimension adaptation
methods = ["project", "pad", "truncate"]
target_dim = 768  # Production transformer dimension

print(f"\nAdapting from {embeddings.shape[-1]}D to {target_dim}D:")
print("-" * 50)

# Loop through each method and test adaptation
for method in methods:
    print(f"\nMethod: {method.upper()}")
    
    try:
        adapted = adapt_dimensions(embeddings, target_dim, method=method)
        print(f"   Original: {embeddings.shape}")
        print(f"   Adapted:  {adapted.shape}")
        print(f"   Successfully adapted using {method} method")
    except Exception as e:
        print(f"   Error with {method} method: {e}")

# Test reverse adaptation (Production → Reference)
print(f"\n" + "=" * 70)
print("Testing reverse adaptation (Production → Reference)")
print("-" * 50)

reference_dim = 64
# Create a simulated production tensor
production_tensor = torch.randn(1, 6, 768)
print(f"Production tensor shape: {production_tensor.shape}")

for method in methods:
    print(f"\nReverse Method: {method.upper()}")
    try:
        reverse_adapted = adapt_dimensions(production_tensor, reference_dim, method=method)
        print(f"   Production: {production_tensor.shape}")
        print(f"   Adapted:    {reverse_adapted.shape}")
        print(f"   Successfully adapted from 768D to 64D using {method}")
    except Exception as e:
        print(f"   Error with reverse {method}: {e}")

print(f"\nREFLECTION:")
print("1. Project method creates learned transformations")
print("2. Pad method preserves original information but adds zeros")
print("3. Truncate method may lose information but is simple")
print("4. Each method has different computational and information trade-offs")

## 🔬 The Big Picture: How Attention Transforms Understanding

You have successfully implemented the attention mechanism! Let's connect all the pieces to see the complete picture.

### The Four-Step Journey

**The attention mechanism solves a fundamental problem**: How can each word in a sentence understand and incorporate information from all other words?

1. **Linear Projections (Q, K, V)**: Create three different "views" of each word
   - Transform static embeddings into dynamic, task-specific representations
   - Enable words to express what they need, what they offer, and what they contain

2. **Scaled Dot-Product**: Measure compatibility between information needs and offerings
   - Quantify relationships through geometric similarity (dot products)
   - Scale to maintain stable gradients for effective learning

3. **Softmax Normalization**: Convert compatibility into attention allocation  
   - Create probability distributions for interpretable attention weights
   - Ensure each word allocates exactly 100% of its attention across all positions

4. **Value Aggregation**: Gather and combine relevant information
   - Perform weighted averaging based on attention decisions
   - Create contextualized representations that incorporate global information

### The Transformation: From Static to Dynamic

**Before Attention** (Static embeddings):
```
"The" → [article, definite, ...]
"cat" → [animal, feline, small, ...]  
"sat" → [action, past, positioning, ...]
"on"  → [preposition, spatial, ...]
"the" → [article, definite, ...]
"mat" → [object, flat, surface, ...]
```

**After Attention** (Contextualized representations):
```
"The" → [article, **refers to cat**, definite, ...]
"cat" → [animal, **performs sitting**, feline, **subject role**, ...]
"sat" → [action, **done by cat**, past, **on surface**, ...]  
"on"  → [preposition, **connects cat and mat**, spatial, ...]
"the" → [article, **refers to mat**, definite, ...]
"mat" → [object, **location of sitting**, flat, **receives cat**, ...]
```

### Key Insights and Implications

#### 1. **Parallel Processing**
Unlike sequential models (RNNs), attention processes all positions simultaneously:
- All words can attend to all other words in one pass
- Enables parallelization and faster training
- Captures long-range dependencies directly

#### 2. **Learned Relationships**  
The attention patterns emerge from learning, not hard-coded rules:
- Q, K, V projections learn what relationships to look for
- Attention weights discover syntactic and semantic patterns
- Model learns grammar, syntax, and semantics implicitly

#### 3. **Context-Dependent Meaning**
Words develop different meanings based on context:
- "bank" in "river bank" vs. "savings bank" gets different attended information
- Same mechanism handles ambiguity resolution and context integration
- Dynamic contextualization at every layer

#### 4. **Foundation for Transformers**
This attention mechanism is the core building block of:
- **BERT**: Bidirectional attention for understanding
- **GPT**: Causal (masked) attention for generation  
- **T5**: Encoder-decoder attention for translation
- **Vision Transformers**: Attention over image patches

### Mathematical Elegance

The entire mechanism can be expressed in one equation:
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

This simple formula encapsulates:
- Information retrieval (queries and keys)
- Relevance measurement (dot products)
- Decision making (softmax)
- Information aggregation (weighted values)

### What You've Learned

1. **Theoretical Foundation**: Deep understanding of why each component is necessary
2. **Mathematical Formulation**: Precise equations and their intuitive meanings  
3. **Implementation Skills**: Hands-on experience building attention from scratch
4. **Tensor Thinking**: Understanding of shapes, dimensions, and operations
5. **Architectural Insight**: How attention enables modern language models

### Next Steps

With this foundation, you're ready to explore:
- **Multi-head attention**: Multiple parallel attention mechanisms
- **Transformer architecture**: Stacking attention with feedforward layers
- **Positional encoding**: Handling sequence order information
- **Advanced variants**: Sparse attention, linear attention, and more

**Congratulations!** You've mastered one of the most important innovations in modern AI. The attention mechanism you've implemented forms the backbone of today's most powerful language models and continues to drive breakthroughs in artificial intelligence.

In [ ]:
# COMPLETED: Explore dimension adaptation

# Import the adapt_dimensions function
from src.model_utils import adapt_dimensions

print("Exploring dimension adaptation between reference and production scales...")
print("=" * 70)

# Get your reference embeddings from earlier in the notebook
print(f"Starting with reference embeddings: {embeddings.shape}")

# Experiment with dimension adaptation
methods = ["project", "pad", "truncate"]
target_dim = 768  # Production transformer dimension

print(f"\nAdapting from {embeddings.shape[-1]}D to {target_dim}D:")
print("-" * 50)

# Loop through each method and test adaptation
for method in methods:
    print(f"\nMethod: {method.upper()}")
    
    try:
        adapted = adapt_dimensions(embeddings, target_dim, method=method)
        print(f"   Original: {embeddings.shape}")
        print(f"   Adapted:  {adapted.shape}")
        print(f"   Successfully adapted using {method} method")
    except Exception as e:
        print(f"   Error with {method} method: {e}")

# Test reverse adaptation (Production → Reference)
print(f"\n" + "=" * 70)
print("Testing reverse adaptation (Production → Reference)")
print("-" * 50)

reference_dim = 64
# Create a simulated production tensor
production_tensor = torch.randn(1, 6, 768)
print(f"Production tensor shape: {production_tensor.shape}")

for method in methods:
    print(f"\nReverse Method: {method.upper()}")
    try:
        reverse_adapted = adapt_dimensions(production_tensor, reference_dim, method=method)
        print(f"   Production: {production_tensor.shape}")
        print(f"   Adapted:    {reverse_adapted.shape}")
        print(f"   Successfully adapted from 768D to 64D using {method}")
    except Exception as e:
        print(f"   Error with reverse {method}: {e}")

print(f"\nREFLECTION:")
print("1. Project method creates learned transformations")
print("2. Pad method preserves original information but adds zeros")
print("3. Truncate method may lose information but is simple")
print("4. Each method has different computational and information trade-offs")

---
# 🎉 Conclusion: Your Journey Through Attention

## What You've Accomplished

Congratulations! You've successfully completed a comprehensive exploration of the attention mechanism. Let's reflect on your journey:

### ✅ Core Implementation (Required)
You've built the fundamental attention mechanism from scratch:
- **Q, K, V Projections**: Created three perspectives on information
- **Scaled Dot-Product**: Measured compatibility between queries and keys
- **Softmax Normalization**: Converted scores to probability distributions
- **Value Aggregation**: Combined information based on attention weights

Your implementation achieved **100% on the evaluation**, demonstrating complete understanding of the core concepts.

### 🔬 Deep Exploration (Optional)
You've explored advanced topics that bridge theory to practice:
- **Dimension Adaptation**: Understood how to scale between educational and production models
- **Production Comparison**: Compared your implementation with DistilGPT-2
- **Architectural Insights**: Discovered that the same mathematical foundation scales from 64D to 768D+

## Key Takeaways

1. **The Same Math, Different Scale**: Your 64D educational implementation and production's 768D transformers use identical mathematical operations - the difference is in scale and optimization.

2. **Attention is Universal**: The mechanism you implemented is the foundation of:
   - Language models (GPT, BERT)
   - Translation systems
   - Computer vision transformers
   - And many more applications

3. **Understanding Fundamentals Matters**: By building attention from scratch, you now understand what's happening inside every transformer model, regardless of size.

## Your Next Steps

With this foundation, you're prepared to explore:

### Immediate Next Steps
- **Multi-Head Attention**: Implement parallel attention mechanisms
- **Positional Encoding**: Add sequence order information
- **Complete Transformer**: Build encoder-decoder architecture

### Advanced Topics
- **Fine-tuning**: Adapt pre-trained models for specific tasks
- **Attention Variants**: Explore sparse, linear, and other efficient attention mechanisms
- **Cross-Modal Applications**: Apply attention to vision, speech, and multimodal tasks

### Practical Applications
- Build a simple chatbot using your attention implementation
- Create a text classifier with transformer architecture
- Experiment with attention visualization on real texts

## Final Thoughts

You've mastered one of the most important innovations in modern AI. The attention mechanism you've implemented is not just an academic exercise - it's the exact same mechanism (at a smaller scale) that powers ChatGPT, BERT, and other state-of-the-art models.

Remember: **You now understand the core technology behind the AI revolution.**

Keep exploring, keep building, and keep learning. The journey from understanding attention to building powerful AI applications is shorter than you might think!

**Well done! 🚀**